# 模型文件转GGUF上传huggingface

1. 下载模型
2. `llama.cpp/convert.py`转化模型权重为GGUF格式
3. `llama.cpp/build/bin/quantize`量化模型权重
4. 上传模型

In [5]:
!pip install huggingface_hub > /dev/null
!git clone https://github.com/ggerganov/llama.cpp.git > /dev/null
!pip install -r llama.cpp/requirements.txt > /dev/null

Cloning into 'llama.cpp'...
remote: Enumerating objects: 22769, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 22769 (delta 0), reused 1 (delta 0), pack-reused 22764
Receiving objects: 100% (22769/22769), 25.70 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (16095/16095), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have to

In [6]:
# Download Model
from huggingface_hub import snapshot_download
model_id="shenzhi-wang/Llama3-8B-Chinese-Chat"
snapshot_download(repo_id=model_id, local_dir="Llama3-8B-Chinese-Chat-hf",
                  local_dir_use_symlinks=False, revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

train_results.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

'/content/Llama3-8B-Chinese-Chat-hf'

In [7]:
# convert hf-model to GGUF, fp32 can help model maintain better performance during quantization process
# llama3 tokenizer type bpe
!python llama.cpp/convert.py Llama3-8B-Chinese-Chat-hf --outtype f16 --outfile "Llama3-8B-Chinese-Chat-fp16.GGUF" --vocab-type bpe > /dev/null

In [8]:
!ls /content

Llama3-8B-Chinese-Chat-fp16.GGUF  Llama3-8B-Chinese-Chat-hf  llama.cpp	sample_data


In [9]:
# compile llama.cpp
%cd llama.cpp
!pwd
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release > /dev/null

/content/llama.cpp
/content/llama.cpp
/content/llama.cpp/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with LLAMA_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Configuring done (1.8s)
-- Generating done (0.2s)
-- Build files have been written

In [18]:
%cd /content
!ls

/content
Llama3-8B-Chinese-Chat-fp16.GGUF  Llama3-8B-Chinese-Chat-hf  llama.cpp	sample_data  upload.py


In [20]:
!ls llama.cpp/build/bin

baby-llama		 lookup-stats		      test-llama-grammar
batched			 main			      test-model-load-cancel
batched-bench		 parallel		      test-quantize-fns
beam-search		 passkey		      test-quantize-perf
benchmark		 perplexity		      test-rope
convert-llama2c-to-ggml  q8dot			      test-sampling
embedding		 quantize		      test-tokenizer-0-falcon
eval-callback		 quantize-stats		      test-tokenizer-0-llama
export-lora		 retrieval		      test-tokenizer-1-aquila
finetune		 save-load-state	      test-tokenizer-1-baichuan
gguf			 server			      test-tokenizer-1-falcon
gguf-split		 simple			      test-tokenizer-1-gpt2
gritlm			 speculative		      test-tokenizer-1-gpt-neox
imatrix			 test-autorelease	      test-tokenizer-1-llama
infill			 test-backend-ops	      test-tokenizer-1-mpt
llama-bench		 test-c			      test-tokenizer-1-refact
llava-cli		 test-chat-template	      test-tokenizer-1-stablelm-3b-4e1t
lookahead		 test-grad0		      test-tokenizer-1-starcoder
lookup			 test-grammar-integration     

In [17]:
%%writefile upload.py

# 上传模型到huggingface
from huggingface_hub import HfApi
import os
import argparse

if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.add_argument("--hf_token")   # huggingface write token
  parser.add_argument("--model_id")   # 你要创建的模型ID 一般是 username/modelname
  parser.add_argument("--local_path") # 上传的本地文件路径
  parser.add_argument("--repo_path")  # repo中的文件路径
  args = parser.parse_args()

  os.environ["HUGGING_FACE_HUB_TOKEN"] = args.hf_token
  api = HfApi()

  model_id = args.model_id
  api.create_repo(model_id, exist_ok=True, repo_type="model")
  api.upload_file(
      path_or_fileobj=args.local_path,
      path_in_repo=args.repo_path,
      repo_id=model_id,
  )

Writing upload.py


In [ ]:
# quantize model with different precision, balance quality, prefer q4_k_m
!llama.cpp/build/bin/quantize Llama3-8B-Chinese-Chat-fp16.GGUF Llama3-8B-Chinese-Chat.q2_k.GGUF q2_k > /dev/null
!python upload.py --hf_token "huggingface write token" \
--model_id "你要创建的模型ID 一般是 username/modelname" \
--local_path "上传的本地文件路径" \
--repo_path "repo中的文件路径"
# !rm -rf Llama3-8B-Chinese-Chat.q2_k.GGUF

main: build = 2714 (e931888d)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'Llama3-8B-Chinese-Chat-fp16.GGUF' to 'Llama3-8B-Chinese-Chat.q2_k.GGUF' as Q2_K
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from Llama3-8B-Chinese-Chat-fp16.GGUF (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_c